## Image sample (DDPM - guided diffusion - Diffusion beats gans)

In [ ]:
# from __future__ import print_function 
import argparse
import os, sys, glob

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as th
import PIL
import copy
import pytorch_lightning as pl
sys.path.insert(0, '../../')
from guided_diffusion.script_util import (
    seed_all,
)
import importlib

# Sample utils
sys.path.insert(0, '../')
from sample_utils import (
    ckpt_utils, 
    params_utils, 
    vis_utils, 
    file_utils, 
    img_utils, 
    inference_utils, 
    mani_utils,
    attr_mani,
)

In [ ]:
# Define loading settings 
# cfg_name = "all_reduce_mult_SharedConv_EncLight_shaded1_adaptivenonzero.yaml"
# log_dir = "all_reduce_mult_SharedConv_EncLight_shaded1_adaptivenonzero"
cfg_name = "all_reduce_nomult_EncLight_adaptivenonzero.yaml"
log_dir = "all_reduce_nomult_EncLight_adaptivenonzero"
set_ = "valid"
seed = 47
seed_all(seed)
if set_ == 'itw':
    img_dataset_path = "../../itw_images/aligned/"
elif set_ == 'train' or set_ == 'valid':
    img_dataset_path = f"/data/mint/ffhq_256_with_anno/ffhq_256/{set_}/"
else: raise NotImplementedError

# Checkpoint loader
step = "500000"
ckpt_selector = "ema"
ckpt_loader = ckpt_utils.CkptLoader(log_dir=log_dir, cfg_name=cfg_name)
cfg = ckpt_loader.cfg
model_dict, diffusion = ckpt_loader.load_model(ckpt_selector=ckpt_selector, step=step)


In [ ]:
if False:
    batch_size = 10
    # Load params
    params_set = params_utils.get_params_set(set=set_, cfg=cfg)

    rand_idx = np.random.choice(a=range(len(list(params_set.keys()))), replace=False, size=batch_size)
    img_path = file_utils._list_image_files_recursively(img_dataset_path)
    img_path = [img_path[r] for r in rand_idx]
    img_name = [path.split('/')[-1] for path in img_path]

    # Load condition & Images into model_kwargs
    model_kwargs = mani_utils.load_condition(params_set, img_name)
    images = mani_utils.load_image(all_path=img_path, cfg=cfg, vis=True)['image']
    model_kwargs.update({'image_name':img_name, 'image':images})
    base_idx = 11
    mode = {'init_noise':'vary_noise', 'cond_params':'vary_cond'}
    interchange = None

    # Input 
    init_noise = inference_utils.get_init_noise(n=batch_size, mode='fixed_noise', img_size=cfg.img_model.image_size, device=ckpt_loader.device)
    cond = model_kwargs.copy()

    cond_enc = {'light':model_dict['ImgEncoder'](cond['image'].cuda().float())}
    cond['light'] = cond_enc['light']

    # Input 
    init_noise = inference_utils.get_init_noise(n=len(img_name), mode='fixed_noise', img_size=cfg.img_model.image_size, device=ckpt_loader.device)

    # Finalize the cond_params
    key_cond_params = mani_utils.without(cfg.param_model.params_selector, cfg.param_model.rmv_params)
    cond = mani_utils.create_cond_params(cond=cond, key=key_cond_params)
    cond = inference_utils.to_tensor(cond, key=['cond_params', 'light', 'image'], device=ckpt_loader.device)

    # Reverse
    pl_reverse_sampling = inference_utils.PLReverseSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_reverse_sample_loop, cfg=cfg)
    reverse_ddim_sample = pl_reverse_sampling(x=cond['image'], model_kwargs=cond)

    # Forward
    pl_sampling = inference_utils.PLSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_sample_loop, cfg=cfg)
    sample_ddim = pl_sampling(noise=reverse_ddim_sample['img_output'], model_kwargs=cond)

    # Visualization
    fig = vis_utils.plot_sample(img=cond['image'], reverse_sampling_images=reverse_ddim_sample['img_output'], sampling_img=sample_ddim['img_output'])
    plt.show()

In [ ]:
src_ref = '61048.jpg'
dst_ref = '61339.jpg'
params_set = params_utils.get_params_set(set=set_, cfg=cfg)
img_name = [src_ref, dst_ref]
model_kwargs = mani_utils.load_condition(params_set, img_name=img_name)
images = mani_utils.load_image_by_name(img_name=img_name, img_dataset_path=img_dataset_path, cfg=cfg, vis=True)['image']
model_kwargs.update({'image_name':img_name, 'image':images})
cond = model_kwargs.copy()
cond['light'] = model_dict['ImgEncoder'](cond['image'].cuda().float())

# Finalize the cond_params
key_cond_params = mani_utils.without(cfg.param_model.params_selector, cfg.param_model.rmv_params)
cond = mani_utils.create_cond_params(cond=cond, key=key_cond_params)
cond = inference_utils.to_tensor(cond, key=['cond_params', 'light', 'image'], device=ckpt_loader.device)

# Reverse
pl_reverse_sampling = inference_utils.PLReverseSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_reverse_sample_loop, cfg=cfg)
reverse_ddim_sample = pl_reverse_sampling(x=cond['image'], model_kwargs=cond)

pl_sampling = inference_utils.PLSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_sample_loop, cfg=cfg)
sample_ddim = pl_sampling(noise=reverse_ddim_sample['img_output'], model_kwargs=cond)

# Visualization
fig = vis_utils.plot_sample(img=cond['image'], reverse_sampling_images=reverse_ddim_sample['img_output'], sampling_img=sample_ddim['img_output'])
plt.show()

# Linear Classifier

In [ ]:
'''
Prepare data
 - Positive class = destination = 1
 - Negative class = source = 0
'''
k = 30
params_set = params_utils.get_params_set(set=set_, cfg=cfg)
# Source = label as 0
src_ref = '61048.jpg'
ref_params = params_set[src_ref]['light']
src_images, src_params_dict = attr_mani.retrieve_topk_params(params_set=params_set, cfg=cfg, ref_params=ref_params, img_dataset_path=img_dataset_path, k=k, dist_type='l2')
src_label = th.zeros(k)
# Destination = label as 1
dst_ref = '61339.jpg'
ref_params = params_set[dst_ref]['light']
dst_images, dst_params_dict = attr_mani.retrieve_topk_params(params_set=params_set, cfg=cfg, ref_params=ref_params, img_dataset_path=img_dataset_path, k=k, dist_type='l2')
dst_label = th.ones(k)

src_params = model_dict['ImgEncoder'](src_images.cuda().float()).detach()
dst_params = model_dict['ImgEncoder'](dst_images.cuda().float()).detach()

input = th.cat((src_params, dst_params), dim=0)
gt = th.cat((src_label, dst_label))[..., None]


In [ ]:
# Train the Linear Classifier
seed_all(47)
cls_model = attr_mani.LinearClassifier(cfg).cuda()
cls_model.train(gt=gt.cuda(), input=input.float().cuda(), n_iters=50000, progress=True)

print("[#] Parameters")
for k, v in cls_model.named_parameters():
    print(k, v, v.shape)

print("[#] Evaluation")
cls_model.evaluate(gt=gt.cuda(), input=input.float().cuda())

print(f"[#] Interpolate Direction = {th.nn.functional.normalize(cls_model.cls.weight, dim=1)}")

In [ ]:
'''
[#] Load condition, image and encoding light
'''
importlib.reload(mani_utils)
img_name = [src_ref, dst_ref]
model_kwargs = mani_utils.load_condition(params_set, img_name=img_name)
images = mani_utils.load_image_by_name(img_name=img_name, img_dataset_path=img_dataset_path, cfg=cfg, vis=True)['image']
model_kwargs.update({'image_name':img_name, 'image':images})
cond = model_kwargs.copy()
cond['light'] = model_dict['ImgEncoder'](cond['image'].cuda().float())

# Finalize the cond_params
key_cond_params = mani_utils.without(cfg.param_model.params_selector, cfg.param_model.rmv_params)
cond = mani_utils.create_cond_params(cond=cond, key=key_cond_params)
cond = inference_utils.to_tensor(cond, key=['cond_params', 'light', 'image'], device=ckpt_loader.device)

'''
Reverse & Forward process
'''
# Reverse
pl_reverse_sampling = inference_utils.PLReverseSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_reverse_sample_loop, cfg=cfg)
reverse_ddim_sample = pl_reverse_sampling(x=cond['image'], model_kwargs=cond)

# Forward
pl_sampling = inference_utils.PLSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_sample_loop, cfg=cfg)
sample_ddim = pl_sampling(noise=reverse_ddim_sample['img_output'], model_kwargs=cond)

# Visualization
fig = vis_utils.plot_sample(img=cond['image'], reverse_sampling_images=reverse_ddim_sample['img_output'], sampling_img=sample_ddim['img_output'])
plt.show()


In [ ]:
'''
[#] With classifier interpolation
'''
importlib.reload(mani_utils)
n = 10
itp_dir = th.nn.functional.normalize(cls_model.cls.weight, dim=1).detach().cpu().numpy()
cond = model_kwargs.copy()
cond['light'] = model_dict['ImgEncoder'](cond['image'].cuda().float())

interp_cond = mani_utils.interp_by_dir(cond.copy(), src_idx=0, itp_name='light', direction=itp_dir, n_step=n)
cond = mani_utils.repeat_cond_params(cond, base_idx=0, n=n, key=mani_utils.without(cfg.param_model.params_selector, ['light']))
cond.update(interp_cond)
key_cond_params = mani_utils.without(cfg.param_model.params_selector, cfg.param_model.rmv_params)
cond = mani_utils.create_cond_params(cond=cond, key=key_cond_params)
cond = inference_utils.to_tensor(cond, key=['cond_params', 'light'], device=ckpt_loader.device)

# Forward with reversed noise
pl_sampling = inference_utils.PLSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_sample_loop, cfg=cfg)
sample_from_rev = pl_sampling(noise=th.cat([reverse_ddim_sample['img_output'][[0]]]*n, dim=0), model_kwargs=cond)

# Forward with N(0, 1) noise
init_noise = inference_utils.get_init_noise(n=n, mode='fixed_noise', img_size=cfg.img_model.image_size, device=ckpt_loader.device)
pl_sampling = inference_utils.PLSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_sample_loop, cfg=cfg)
sample_from_gauss = pl_sampling(noise=init_noise, model_kwargs=cond)

fig = vis_utils.plot_sample(img=sample_from_rev['img_output'])
plt.show()
fig = vis_utils.plot_sample(img=sample_from_gauss['img_output'])
plt.show()

In [ ]:
'''
[#] Without classifier interpolation
'''
cond = model_kwargs.copy()
cond['light'] = model_dict['ImgEncoder'](cond['image'].cuda().float())

interp_cond = mani_utils.iter_interp_cond(cond.copy(), interp_set=['light'], src_idx=0, dst_idx=1, n_step=n)
cond = mani_utils.repeat_cond_params(cond, base_idx=0, n=n, key=mani_utils.without(cfg.param_model.params_selector, ['light']))
cond.update(interp_cond)

# Finalize the cond_params
key_cond_params = mani_utils.without(cfg.param_model.params_selector, cfg.param_model.rmv_params)
cond = mani_utils.create_cond_params(cond=cond, key=key_cond_params)
cond = inference_utils.to_tensor(cond, key=['cond_params', 'light'], device=ckpt_loader.device)

# Forward
pl_sampling = inference_utils.PLSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_sample_loop, cfg=cfg)
sample_from_rev = pl_sampling(noise=th.cat([reverse_ddim_sample['img_output'][[0]]]*n, dim=0), model_kwargs=cond)

# Forward with N(0, 1) noise
init_noise = inference_utils.get_init_noise(n=n, mode='fixed_noise', img_size=cfg.img_model.image_size, device=ckpt_loader.device)
pl_sampling = inference_utils.PLSampling(model_dict=model_dict, diffusion=diffusion, sample_fn=diffusion.ddim_sample_loop, cfg=cfg)
sample_from_gauss = pl_sampling(noise=init_noise, model_kwargs=cond)

fig = vis_utils.plot_sample(img=sample_from_rev['img_output'])
plt.show()
fig = vis_utils.plot_sample(img=sample_from_gauss['img_output'])
plt.show()